In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import torch
import pytorch_lightning as pl
import numpy as np
import matplotlib.pyplot as plt
from timm.utils.lightning_train import LitTimm # WARNING: This file is COPIED from the one used for actual training
from pathlib import Path

In [16]:
# logdir = Path("/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/")
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT01/tavt_newatt_nobiases_weightsum/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)

In [17]:
model.model.blocks.block.attn.weight_sum

Parameter containing:
tensor([-0.2025, -0.3117, -0.2029, -0.1456, -0.2612, -0.1953, -0.2297, -0.5367,
        -0.1950, -0.2246, -0.6338, -0.2383], requires_grad=True)

Analysis plan

- Analyze the weighted sums trained over the weekend
- Compare with K=V , norm of the value matrix
- Check values of kqt1

I will need 2 that work really well, and 2 that kinda work but are our method

What if one (both) of the energy terms are supposed to be negative?
    - Wouldn't the model just learn the negative values themselves?



In [11]:
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT02/tavt_newatt_weightsum_eneg4clipinit-True/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)
model.model.blocks.block.attn.weight_sum

Parameter containing:
tensor([-4.0361e-03, -8.6164e-03, -4.9885e-03, -4.4652e-01, -5.5194e-01,
         8.9605e-01, -3.6049e-03, -5.1600e-03, -6.5746e-01, -1.7637e-03,
        -5.8060e-05, -5.6285e-03], requires_grad=True)

In [15]:
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT02/tavt_newatt_weightsum_eneg3clipinit-False/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)
model.model.blocks.block.attn.weight_sum.max()

tensor(0.4455, grad_fn=<MaxBackward1>)

In [14]:
ckpt = "/gpfs/u/home/DAMT/DAMThvrb/scratch/tb_logs/TAVT02/tavt_newatt_weightsum_eneg3clipinit-True/checkpoints/last.ckpt"
model = LitTimm.load_from_checkpoint(ckpt)
model.model.blocks.block.attn.weight_sum.max()

tensor(0.4931, grad_fn=<MaxBackward1>)

In [32]:
no_headweight_training=True
def filter_k(k):
    c1 = "betas" not in k
    if no_headweight_training:
        c2 = "weight_sum" not in k
        return c1 and c2
    return c1

In [33]:
params = [k for k, v in model.model.named_parameters() if filter_k(k)]


In [34]:
len(params)

16